In [1]:
from pymorphy2 import MorphAnalyzer
import pandas as pd
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import shap
import sklearn
import os

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest 
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import SGDRegressor
from scipy.stats import probplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.stattools import durbin_watson
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
from catboost import cv
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from catboost.utils import create_cd
from catboost.eval.catboost_evaluation import *
from catboost.eval.evaluation_result import *
from catboost.utils import get_roc_curve
from sklearn import metrics
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [19]:
file_path = 'test_data.csv'

# Чтение данных из файла CSV и запись их в датафрейм
df = pd.read_csv(file_path)
# df[['month', 'year']] = df['Дата платежа'].str.split(' ', expand=True)

# # Преобразование значений 'month' и 'year' в числовой тип данных (int)
# df['month'] = df['month'].astype(int)
# df['year'] = df['year'].astype(int)

In [25]:
df['Отнесено'] = 48642.16378704082

In [27]:
df.to_csv('test_out.csv')

In [9]:
payment = pd.read_excel('payment.xlsx')
payment[['day', 'month', 'year']] = payment['Дата проведения'].str.split('.', expand=True)

# Преобразование значений 'day', 'month' и 'year' в числовой тип данных (int)
payment['day'] = payment['day'].astype(int)
payment['month'] = payment['month'].astype(int)
payment['year'] = payment['year'].astype(int)
payment['year'] += 2

In [23]:
payment['Конечный остаток'].mean()

48642.16378704082

In [13]:
# Создаем экземпляр MorphAnalyzer
morph = MorphAnalyzer()

def preprocess_text(text):
    # Разделяем текст на слова, нормализуем каждое слово и возвращаем множество нормализованных форм
    return set(morph.parse(word)[0].normal_form for word in text.split())

# Создание нового столбца с нормализованными формами слов в столбце "Название"
df['normalized_names'] = df['Направление расходов'].apply(preprocess_text)

# Показываем первые строки для проверки
print(df.head())

  Наименование вида расходов  \
0             Электроэнергия   
1             Электроэнергия   
2             Электроэнергия   
3             Электроэнергия   
4             Электроэнергия   

                                Направление расходов Дата платежа  Отнесено  \
0  134.1000.01 ДГУ. Оплата за электроэнергию по т...      03 2022       NaN   
1  134.1000.01 ДГУ. Оплата за электроэнергию по т...      04 2022       NaN   
2  134.1000.01 ДГУ. Оплата за электроэнергию по т...      05 2022       NaN   
3  134.1000.01 ДГУ. Оплата за электроэнергию по т...      06 2022       NaN   
4  134.1000.01 ДГУ. Оплата за электроэнергию по т...      07 2022       NaN   

   month  year                                   normalized_names  
0      3  2022  {оплата, за, 6, ул.мордовцева,, 134.1000.01, п...  
1      4  2022  {оплата, за, 6, ул.мордовцева,, 134.1000.01, п...  
2      5  2022  {оплата, за, 6, ул.мордовцева,, 134.1000.01, п...  
3      6  2022  {оплата, за, 6, ул.мордовцева,, 134.1000.01,

In [16]:
import pandas as pd
from tqdm import tqdm

# Функция для расчета коэффициента Жаккара
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Добавляем столбец для Конечного остатка в df
df['Отнесено'] = None

# Итерация по каждой строке в df
for i, df_row in tqdm(df.iterrows(), total=df.shape[0]):
    best_match_ratio = 0
    best_final_balance = None
    
    # Итерация по каждой строке в payment
    for j, payment_row in payment.iterrows():
        # Проверяем совпадение по 'month' и 'year'
        if df_row['month'] == payment_row['month'] and df_row['year'] == payment_row['year']:
            # Считаем коэффициент Жаккара
            ratio = jaccard_similarity(set(df_row['normalized_names']), set(payment_row['normalized_names']))
            
            # Если текущий коэффициент лучше предыдущего
            if ratio > best_match_ratio:
                best_match_ratio = ratio
                best_final_balance = payment_row['Конечный остаток']
    
    # Обновляем значение Отнесено в df
    df.at[i, 'Отнесено'] = best_final_balance



  1%|          | 34/4512 [00:37<1:22:34,  1.11s/it]


KeyboardInterrupt: 

In [ ]:
# df.to_csv('test_out.csv')

In [33]:
test = pd.read_csv('test_out2.csv')

In [34]:
test

,Unnamed: 0,Наименование вида расходов,Направление расходов,Дата платежа,Отнесено,month,year,normalized_names
0,0,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,03 2022,180.18,3,2022,"{'за', '6', 'дгу.', '134.1000.01', 'электроэне..."
1,1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,04 2022,163.80,4,2022,"{'за', '6', 'дгу.', '134.1000.01', 'электроэне..."
2,2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,05 2022,163.80,5,2022,"{'за', '6', 'дгу.', '134.1000.01', 'электроэне..."
3,3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,06 2022,180.18,6,2022,"{'за', '6', 'дгу.', '134.1000.01', 'электроэне..."
4,4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,07 2022,98.28,7,2022,"{'за', '6', 'дгу.', '134.1000.01', 'электроэне..."
...,...,...,...,...,...,...,...,...
4507,4507,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,1,2023,"{'ул.', 'якутия', 'киров', 'потребление', '17б..."
4508,4508,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,1,2023,"{'ул.', '(литерб)', 'якутия', 'киров', 'потреб..."
4509,4509,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,1,2023,"{'ул.', '(литера6)', 'якутия', 'киров', 'потре..."
4510,4510,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN,1,2023,"{'ул.', 'якутия', 'киров', 'потребление', '17м..."


In [40]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

with open('dataframe.pkl', 'rb') as file:
    pred_df = pickle.load(file)

with open('model_el.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
y_pred_el = loaded_model.predict(pred_df)

result_el = {
    '2022':y_pred_el[0:11].sum(),
    '2023':y_pred_el[12:23].sum(),
    '2024':y_pred_el[24:35].sum()
}
with open('model_teplo.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
y_pred_teplo = loaded_model.predict(pred_df)
result_teplo = {
    '2022':y_pred_teplo[0:11].sum(),
    '2023':y_pred_teplo[12:23].sum(),
    '2024':y_pred_teplo[24:35].sum()
}
with open('model_gaz.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

y_pred_gaz = loaded_model.predict(pred_df)
result_gaz = {
    '2022':y_pred_gaz[0:11].sum(),
    '2023':y_pred_gaz[12:23].sum(),
    '2024':y_pred_gaz[24:35].sum()
}
with open('model_voda.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
y_pred_voda = loaded_model.predict(pred_df)

result_voda = {
    '2022':y_pred_voda[0:11].sum(),
    '2023':y_pred_voda[12:23].sum(),
    '2024':y_pred_voda[24:35].sum()
}

total_payments = y_pred_el + y_pred_teplo + y_pred_gaz + y_pred_voda
result = {
    '2022':total_payments[0:11].sum(),
    '2023':total_payments[12:23].sum(),
    '2024':total_payments[24:35].sum()
}

In [41]:
import pandas as pd

# предположим, что test - это ваш DataFrame, и result - ваш словарь
result = {'2022': 2834186.0809283117, '2023': 3030804.1760103996, '2024': 2950727.980900251}

# Преобразуем год в строку для соответствия ключам в словаре
test['year'] = test['year'].astype(str)

# Функция для добавления значения
def add_value(row):
    year = row['year']
    if year in result:
        return row['Отнесено'] + result[year] / 100
    else:
        return row['Отнесено']  # Если год не найден в словаре, возвращаем исходное значение

# Применяем функцию к датафрейму
test['Отнесено'] = test.apply(add_value, axis=1)


In [45]:
import pandas as pd

# предположим, что test - это ваш DataFrame
# Удаляем столбцы 'month', 'year', 'normalized_names'
test = test.drop(columns=['Unnamed: 0', 'month', 'year', 'normalized_names'])


In [46]:
test.to_csv("test_out4.csv")